#Adding in city_id & city for each neighborhood (6.16)
###Purpose: 
Add the city_id & city fields to each record in the MongoDB neighborhoods collection.

###Background: 
On my first loop through AirBnB, I didn't save the city_id or city fields from the AirBnBNeighborhood class to MongoDB. I updated the code in my class, but would rather not rescrape everything.

In [2]:
import pandas as pd
from pymongo import MongoClient

db_name = 'airbnb'
coll_name = 'neighborhoods'

client = MongoClient()
db = client[db_name]
coll = db[coll_name]

In [34]:
df = pd.read_csv('../data/neighborhood_list.csv')
df.head()

,neighborhood_id,neighborhood,neighborhood_url,city_id,city
0,0,Alamo Square,/locations/san-francisco/alamo-square,1,san-francisco
1,1,Bayview,/locations/san-francisco/bayview,1,san-francisco
2,2,Bernal Heights,/locations/san-francisco/bernal-heights,1,san-francisco
3,3,Chinatown,/locations/san-francisco/chinatown,1,san-francisco
4,4,Civic Center,/locations/san-francisco/civic-center,1,san-francisco


In [57]:
import airbnbneighborhood as airn
reload(airn)
air_hood = airn.AirBnBNeighborhood(db_name = db_name, coll_name = coll_name)
air_hood

In [41]:
for hood in df[['neighborhood_id', 'city_id','city']].to_dict(orient='records'):
    hood_id = hood.pop('neighborhood_id')
    air_hood.pull_from_db(neighborhood_id = hood_id)
    air_hood.add_features(new_features=hood)
    # print air_hood.neighborhood, hood

In [56]:
#confirming
for i in df['neighborhood_id']:
    air_hood.pull_from_db(i)
    print air_hood.d['city']

san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-yor

#Creating a mapping table for ammenities (6.16)
###Purpose: 
Create a mapping table from AirBnB's amenity_ids (from the meta data) to the actual amenity.

###Background: 
In the listing, AirBnB provides a list of amenities offered by the listing. However, they don't explicitly state the ammenity. This is meant to create a mapping table from the amenity_id to the amenity.

In [58]:
import airbnblisting as airl
from bs4 import BeautifulSoup
import json

In [59]:
reload(airl)
DB_NAME = 'airbnb'
COLL_NAME = 'listings'

air_listing = airl.AirBnBListing(db_name=DB_NAME, coll_name=COLL_NAME)
air_listing

In [60]:
sample_id = '470028'
air_listing.pull_from_db(listing_id = sample_id)
air_listing.url

u'https://www.airbnb.com/rooms/470028'

In [61]:
soup = BeautifulSoup(air_listing.r.content)

In [94]:
list_of_ammenities = []
for x in soup.find('div', {'class':"expandable-content-full"}).find_all('span'):
    list_of_ammenities.append(\
                              {'id':int(x['id'][::-1][:x['id'][::-1].find('-')][::-1]),
                               'amenity':x.get_text().strip()})
df = pd.DataFrame(list_of_ammenities)
df = df.set_index('id').sort()
df.head()
df.to_csv('../data/amenities.csv', index_label='id')